In [1]:
!pip -q install pyshp
!pip -q install influxdb
!pip -q install shapely
!pip -q install ipyleaflet
!pip -q install plotly
!pip -q install folium

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import tarfile
from six.moves import urllib
from shapely.geometry import Point, Polygon
from dotenv import load_dotenv
import glob,os
import pylab as pl
import folium
import shapefile
import json
from influxdb import DataFrameClient
from ipyleaflet import Map, GeoJSON, basemaps, Polyline, Heatmap
import requests
from datetime import datetime, timedelta, date
from collections import OrderedDict
import folium.plugins as plugins
import branca.colormap as cm
import time

load_dotenv()

In [3]:
!jupyter nbextension install --py --symlink --sys-prefix ipyleaflet
!jupyter nbextension enable --py --sys-prefix ipyleaflet

Installing /opt/conda/lib/python3.7/site-packages/ipyleaflet/static -> jupyter-leaflet
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable ipyleaflet --py --sys-prefix
    
Enabling notebook extension jupyter-leaflet/extension...
      - Validating: OK


# Change "variable" for changing measurements

In [4]:
variable = "temperature"

In [5]:
def rm_outside_polygon(res_pd_frame,polygon):
    res_pd_frame_1 = res_pd_frame[['lon','lat','sensor_id']]
    res_pd_frame_2 = res_pd_frame_1.reset_index()
    res_pd_frame_3 = res_pd_frame_2.drop(columns=['index'])
    res_pd_frame4 = res_pd_frame_3.drop_duplicates()
    res_pd_frame4["polygon1"] = res_pd_frame4.apply(lambda row: Polygon(poly).intersects(Point(row["lon"], row["lat"])), axis = 1)
    res_pd_frame5 = res_pd_frame4.drop(res_pd_frame4[res_pd_frame4["polygon1"]].index)
    res_pd_frame = res_pd_frame[~res_pd_frame.sensor_id.isin(res_pd_frame5.sensor_id)]     
    return res_pd_frame

In [7]:
#sf = shapefile.Reader("landkreise-in-germany.shp") # This is Göttingen
#s = sf.shape(0)

sf = shapefile.Reader("bundesland.shp") # Complete germany, 16 shapes for each bundesland
s = sf.shape(3) # Baden-Württemberg
poly = Polygon(s.points)  # Convert to polygon to use shapely function point.within(polygon)

geo_json = GeoJSON(data=s.__geo_interface__, style = {'color': 'blue', 'opacity':0.1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1})

In [8]:
def style_function(feature):
    return {
        'fillOpacity': 0.0,
        'weight': 1.0,
        'fillColor': '#ffffff'
    }

In [9]:
def main(host=os.getenv("HOST"), port=os.getenv("PORT"), rectangle=[0,0,0,0],time_range=["",""],limit=1000):
    """Instantiate the connection to the InfluxDB client."""
    user = os.getenv("USER")
    password = os.getenv("PASSWORD")
    dbname = os.getenv("DB")
    protocol = 'line'

    client = DataFrameClient(host, port, user, password, dbname, ssl=True, verify_ssl=True)

    bind_params = {
        "start_time_epoch": time_range[0],
        "end_time_epoch": time_range[1]
    }
        
    measurement = variable + ", lat, lon, sensor_id"
    
    query = f"select {measurement} from sensor where \
                     time > $start_time_epoch AND \
                     time < $end_time_epoch AND \
                     lon > {rectangle[0]} AND \
                     lon < {rectangle[2]} AND \
                     lat > {rectangle[1]} AND \
                     lat < {rectangle[3]} \
                     limit {limit}"
               
    res=client.query(query, bind_params=bind_params)
    
    return res

In [10]:
appended_new_res_pd_frame_drop_x_groupby = []
locations=[]

start_date = datetime(2017, 1, 2)
end_date = start_date + timedelta(hours=1)

for i in range(0,24):
    time_range=[start_date.strftime("%Y-%m-%dT%H:%M:%SZ"),end_date.strftime("%Y-%m-%dT%H:%M:%SZ")]
    print(time_range)
    
    res=main(rectangle=s.bbox,time_range=time_range,limit=10000)
           
    res_pd_frame=rm_outside_polygon(res['sensor'],poly)
    
    if variable in res_pd_frame.keys():
            res_pd_frame.dropna(subset=[variable],inplace = True)
    
    
    res_pd_frame_mean = res_pd_frame.groupby(['lat','lon', 'sensor_id'],as_index=True)[variable].mean()
    res_pd_frame_mean = res_pd_frame_mean.reset_index()
    
    merge_res_pd_frame_mean = res_pd_frame_mean.loc[:, ['sensor_id', variable]].copy()
    new_res_pd_frame = pd.merge(res_pd_frame, merge_res_pd_frame_mean, on='sensor_id', how='outer')
    
    new_res_pd_frame_drop_x = new_res_pd_frame.drop(columns = [variable + '_x'])
    new_res_pd_frame_drop_x
    
    new_res_pd_frame_drop_x_groupby = new_res_pd_frame_drop_x.groupby(['sensor_id']).mean()
    new_res_pd_frame_drop_x_groupby = new_res_pd_frame_drop_x_groupby.reset_index()
    
    for j in range(0,len(new_res_pd_frame_drop_x_groupby.index)):
        new_res_pd_frame_drop_x_groupby = new_res_pd_frame_drop_x_groupby.rename(index={j:i})
    
    appended_new_res_pd_frame_drop_x_groupby.append(new_res_pd_frame_drop_x_groupby)
    
    start_date=end_date
    end_date=start_date + timedelta(hours=1)
    
    #if start_date == date_to_end_query: 
        #break

appended_new_res_pd_frame_drop_x_groupby = pd.concat(appended_new_res_pd_frame_drop_x_groupby)
appended_new_res_pd_frame_drop_x_groupby.index.name = 'iterate'
a = appended_new_res_pd_frame_drop_x_groupby.reset_index()
a

for jj in range(0,i):
    locations.append(a[a['iterate'] == jj][['lat', 'lon', variable + '_y']].values.tolist())   

map = folium.Map([new_res_pd_frame_drop_x_groupby['lat'].mean(), new_res_pd_frame_drop_x_groupby['lon'].mean()], center=(51.5167, 9.9167), zoom_start=3)

colors = []
folium.GeoJson(
    data=s.__geo_interface__,
    style_function=style_function
    ).add_to(map)

map.fit_bounds([[48, 8],[48, 10],[50, 8],[50, 10]])

hm = plugins.HeatMapWithTime(locations, radius=20, gradient={0.01: 'white', .1: 'blue', .4: 'cyan', .65: 'lime', 1: 'yellow'})
hm.add_to(map)

max_value = a[variable + '_y'].max()
min_value = a[variable + '_y'].min()

linear = cm.LinearColormap(
    ['white', 'cyan', 'yellow'],
    vmin=min_value, vmax=max_value
)


map.add_child(linear)
map



['2017-01-02T00:00:00Z', '2017-01-02T01:00:00Z']


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


['2017-01-02T01:00:00Z', '2017-01-02T02:00:00Z']
['2017-01-02T02:00:00Z', '2017-01-02T03:00:00Z']
['2017-01-02T03:00:00Z', '2017-01-02T04:00:00Z']
['2017-01-02T04:00:00Z', '2017-01-02T05:00:00Z']
['2017-01-02T05:00:00Z', '2017-01-02T06:00:00Z']
['2017-01-02T06:00:00Z', '2017-01-02T07:00:00Z']
['2017-01-02T07:00:00Z', '2017-01-02T08:00:00Z']
['2017-01-02T08:00:00Z', '2017-01-02T09:00:00Z']
['2017-01-02T09:00:00Z', '2017-01-02T10:00:00Z']
['2017-01-02T10:00:00Z', '2017-01-02T11:00:00Z']
['2017-01-02T11:00:00Z', '2017-01-02T12:00:00Z']
['2017-01-02T12:00:00Z', '2017-01-02T13:00:00Z']
['2017-01-02T13:00:00Z', '2017-01-02T14:00:00Z']
['2017-01-02T14:00:00Z', '2017-01-02T15:00:00Z']
['2017-01-02T15:00:00Z', '2017-01-02T16:00:00Z']
['2017-01-02T16:00:00Z', '2017-01-02T17:00:00Z']
['2017-01-02T17:00:00Z', '2017-01-02T18:00:00Z']
['2017-01-02T18:00:00Z', '2017-01-02T19:00:00Z']
['2017-01-02T19:00:00Z', '2017-01-02T20:00:00Z']
['2017-01-02T20:00:00Z', '2017-01-02T21:00:00Z']
['2017-01-02T21:00:0

In [12]:
map.save('Heatmap.html')